# Ethno

This notebook is a global template for loading and preprocess data.

## What is the SVM algorithm?

 It is a supervised learning algorithm designed to solve discrimination and regression problems.
 
 It is a very good image classification algorithm.
 
 ![enter image description here](https://editor.analyticsvidhya.com/uploads/61706svm3.png)
 
 ## Summary
 
 1. [Data preparation](#prepaData)
 2. [Learning of model](#model)
 3. [Displaying metrics](#metric)
 4. [Conclusion](#conclusion)

### 1. Data preparation <a id="prepaData"></a>
 
 - To start, we import the necessary libraries.

 #### Datasets : Download the CUB datasets at this address https://deepai.org/dataset/cub-200-2011 then rename it to 'data'

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import cv2
import time
import shutil
import pickle
import pathlib
import itertools
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dense, Conv2D, Input, Flatten, Dropout, MaxPooling2D
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import load_breast_cancer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import plot_confusion_matrix
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

ModuleNotFoundError: No module named 'cv2'

## data set creation

In [ ]:
ROOT_DIR = './data'


In [ ]:
orig_images_folder = 'images'
if os.path.exists('./data/datasets'):
  shutil.rmtree('./data/datasets')
else:
  new_images_folder = 'datasets'

  image_fnames = pd.read_csv(filepath_or_buffer=os.path.join(ROOT_DIR, 'images.txt'),
                           header=None,
                           delimiter=' ',
                           nrows=543,
                           names=['Img ID', 'file path'])

  image_fnames['is training image?'] = pd.read_csv(filepath_or_buffer=os.path.join(ROOT_DIR, 'train_test_split.txt'),
                                                 header=None, delimiter=' ', nrows=543,
                                                 names=['Img ID', 'is training image?'])['is training image?']
  image_fnames.head()


In [ ]:
if os.path.exists('./data/datasetsSelect'):
   shutil.rmtree('./data/datasetsSelect')

data_dir = os.path.join(ROOT_DIR, orig_images_folder)
new_data_dir = os.path.join(ROOT_DIR, new_images_folder)
os.makedirs(os.path.join(new_data_dir, 'train'), exist_ok=True)
os.makedirs(os.path.join(new_data_dir, 'test'), exist_ok=True)
os.makedirs(os.path.join(new_data_dir, 'val'), exist_ok=True)

for i_image, image_fname in enumerate(image_fnames['file path']):
    if image_fnames['is training image?'].iloc[i_image]:
        new_dir = os.path.join(new_data_dir, 'train',
                               image_fname.split('/')[0])
        os.makedirs(new_dir, exist_ok=True)
        shutil.copy(src=os.path.join(data_dir, image_fname),
                    dst=os.path.join(new_dir, image_fname.split('/')[1]))
    else:
        new_dir = os.path.join(new_data_dir, 'test', image_fname.split('/')[0])
        os.makedirs(new_dir, exist_ok=True)
        shutil.copy(src=os.path.join(data_dir, image_fname),
                    dst=os.path.join(new_dir, image_fname.split('/')[1]))
        
        new_dir = os.path.join(new_data_dir, 'val', image_fname.split('/')[0])
        os.makedirs(new_dir, exist_ok=True)
        shutil.copy(src=os.path.join(data_dir, image_fname),
                    dst=os.path.join(new_dir, image_fname.split('/')[1]))


In [ ]:
# src_test_pigeon = './data/datasets/test/058.Pigeon_Guillemot'
# dest_test_pigeon = './data/datasetsSelect/test/058.Pigeon_Guillemot'

# src_train_pigeon = './data/datasets/train/058.Pigeon_Guillemot'
# dest_train_pigeon = './data/datasetsSelect/train/058.Pigeon_Guillemot'

# src_val_pigeon = './data/datasets/val/058.Pigeon_Guillemot'
# dest_val_pigeon = './data/datasetsSelect/val/058.Pigeon_Guillemot'

# src_test_woodpecker = './data/datasets/test/191.Red_headed_Woodpecker'
# dest_test_woodpecker = './data/datasetsSelect/test/191.Red_headed_Woodpecker'

# src_val_woodpecker = './data/datasets/val/191.Red_headed_Woodpecker'
# dest_val_woodpecker = './data/datasetsSelect/val/191.Red_headed_Woodpecker'

# src_train_woodpecker = './data/datasets/train/191.Red_headed_Woodpecker'
# dest_train_woodpecker = './data/datasetsSelect/train/191.Red_headed_Woodpecker'

src_test_footed_albatross = './data/datasets/test/001.Black_footed_Albatross'
dest_test_footed_albatross = './data/datasetsSelect/test/001.Black_footed_Albatross'

src_train_footed_albatross = './data/datasets/train/001.Black_footed_Albatross'
dest_train_footed_albatross = './data/datasetsSelect/train/001.Black_footed_Albatross'

src_val_footed_albatross = './data/datasets/val/001.Black_footed_Albatross'
dest_val_footed_albatross = './data/datasetsSelect/val/001.Black_footed_Albatross'


src_test_laysan_albatross = './data/datasets/test/002.Laysan_Albatross'
dest_test_laysan_albatross = './data/datasetsSelect/test/002.Laysan_Albatross'

src_val_laysan_albatross = './data/datasets/val/002.Laysan_Albatross'
dest_val_laysan_albatross = './data/datasetsSelect/val/002.Laysan_Albatross'

src_train_laysan_albatross = './data/datasets/train/002.Laysan_Albatross'
dest_train_laysan_albatross = './data/datasetsSelect/train/002.Laysan_Albatross'


src_test_sooty_albatross = './data/datasets/test/003.Sooty_Albatross'
dest_test_sooty_albatross = './data/datasetsSelect/test/003.Sooty_Albatross'

src_val_sooty_albatross = './data/datasets/val/003.Sooty_Albatross'
dest_val_sooty_albatross = './data/datasetsSelect/val/003.Sooty_Albatross'

src_train_sooty_albatross = './data/datasets/train/003.Sooty_Albatross'
dest_train_sooty_albatross = './data/datasetsSelect/train/003.Sooty_Albatross'


src_test_groove_billed_ani = './data/datasets/test/004.Groove_billed_Ani'
dest_test_groove_billed_ani = './data/datasetsSelect/test/004.Groove_billed_Ani'

src_val_groove_billed_ani = './data/datasets/val/004.Groove_billed_Ani'
dest_val_groove_billed_ani = './data/datasetsSelect/val/004.Groove_billed_Ani'

src_train_groove_billed_ani = './data/datasets/train/004.Groove_billed_Ani'
dest_train_groove_billed_ani = './data/datasetsSelect/train/004.Groove_billed_Ani'


src_test_crested_auklet = './data/datasets/test/005.Crested_Auklet'
dest_test_crested_auklet = './data/datasetsSelect/test/005.Crested_Auklet'

src_val_crested_auklet = './data/datasets/val/005.Crested_Auklet'
dest_val_crested_auklet = './data/datasetsSelect/val/005.Crested_Auklet'

src_train_crested_auklet = './data/datasets/train/005.Crested_Auklet'
dest_train_crested_auklet = './data/datasetsSelect/train/005.Crested_Auklet'


src_test_least_auklet = './data/datasets/test/006.Least_Auklet'
dest_test_least_auklet = './data/datasetsSelect/test/006.Least_Auklet'

src_val_least_auklet = './data/datasets/val/006.Least_Auklet'
dest_val_least_auklet = './data/datasetsSelect/val/006.Least_Auklet'

src_train_least_auklet = './data/datasets/train/006.Least_Auklet'
dest_train_least_auklet = './data/datasetsSelect/train/006.Least_Auklet'


src_test_parakeet_auklet = './data/datasets/test/007.Parakeet_Auklet'
dest_test_parakeet_auklet = './data/datasetsSelect/test/007.Parakeet_Auklet'

src_val_parakeet_auklet = './data/datasets/val/007.Parakeet_Auklet'
dest_val_parakeet_auklet = './data/datasetsSelect/val/007.Parakeet_Auklet'

src_train_parakeet_auklet = './data/datasets/train/007.Parakeet_Auklet'
dest_train_parakeet_auklet = './data/datasetsSelect/train/007.Parakeet_Auklet'


src_test_rhinoceros_auklet = './data/datasets/test/008.Rhinoceros_Auklet'
dest_test_rhinoceros_auklet = './data/datasetsSelect/test/008.Rhinoceros_Auklet'

src_val_rhinoceros_auklet = './data/datasets/val/008.Rhinoceros_Auklet'
dest_val_rhinoceros_auklet = './data/datasetsSelect/val/008.Rhinoceros_Auklet'

src_train_rhinoceros_auklet = './data/datasets/train/008.Rhinoceros_Auklet'
dest_train_rhinoceros_auklet = './data/datasetsSelect/train/008.Rhinoceros_Auklet'


src_test_brewer_blackbird = './data/datasets/test/009.Brewer_Blackbird'
dest_test_brewer_blackbird = './data/datasetsSelect/test/009.Brewer_Blackbird'

src_val_brewer_blackbird = './data/datasets/val/009.Brewer_Blackbird'
dest_val_brewer_blackbird = './data/datasetsSelect/val/009.Brewer_Blackbird'

src_train_brewer_blackbird = './data/datasets/train/009.Brewer_Blackbird'
dest_train_brewer_blackbird = './data/datasetsSelect/train/009.Brewer_Blackbird'


src_test_red_winged_blackbird = './data/datasets/test/010.Red_winged_Blackbird'
dest_test_red_winged_blackbird = './data/datasetsSelect/test/010.Red_winged_Blackbird'

src_val_red_winged_blackbird = './data/datasets/val/010.Red_winged_Blackbird'
dest_val_red_winged_blackbird = './data/datasetsSelect/val/010.Red_winged_Blackbird'

src_train_red_winged_blackbird = './data/datasets/train/010.Red_winged_Blackbird'
dest_train_red_winged_blackbird = './data/datasetsSelect/train/010.Red_winged_Blackbird'


shutil.move(src_test_footed_albatross, dest_test_footed_albatross)
shutil.move(src_train_footed_albatross, dest_train_footed_albatross)
shutil.move(src_val_footed_albatross, dest_val_footed_albatross)

shutil.move(src_test_laysan_albatross, dest_test_laysan_albatross)
shutil.move(src_train_laysan_albatross, dest_train_laysan_albatross)
shutil.move(src_val_laysan_albatross, dest_val_laysan_albatross)

shutil.move(src_test_sooty_albatross, dest_test_sooty_albatross)
shutil.move(src_train_sooty_albatross, dest_train_sooty_albatross)
shutil.move(src_val_sooty_albatross, dest_val_sooty_albatross)

shutil.move(src_test_groove_billed_ani, dest_test_groove_billed_ani)
shutil.move(src_train_groove_billed_ani, dest_train_groove_billed_ani)
shutil.move(src_val_groove_billed_ani, dest_val_groove_billed_ani)

shutil.move(src_test_crested_auklet, dest_test_crested_auklet)
shutil.move(src_train_crested_auklet, dest_train_crested_auklet)
shutil.move(src_val_crested_auklet, dest_val_crested_auklet)


shutil.move(src_test_least_auklet, dest_test_least_auklet)
shutil.move(src_train_least_auklet, dest_train_least_auklet)
shutil.move(src_val_least_auklet, dest_val_least_auklet)

shutil.move(src_val_parakeet_auklet, dest_val_parakeet_auklet)
shutil.move(src_test_parakeet_auklet, dest_test_parakeet_auklet)
shutil.move(src_train_parakeet_auklet, dest_train_parakeet_auklet)

shutil.move(src_val_rhinoceros_auklet, dest_val_rhinoceros_auklet)
shutil.move(src_test_rhinoceros_auklet, dest_test_rhinoceros_auklet)
shutil.move(src_train_rhinoceros_auklet, dest_train_rhinoceros_auklet)


shutil.move(src_val_brewer_blackbird, dest_val_brewer_blackbird)
shutil.move(src_test_brewer_blackbird, dest_test_brewer_blackbird)
shutil.move(src_train_brewer_blackbird, dest_train_brewer_blackbird)


shutil.move(src_val_red_winged_blackbird, dest_val_red_winged_blackbird)
shutil.move(src_test_red_winged_blackbird, dest_test_red_winged_blackbird)
shutil.move(src_train_red_winged_blackbird, dest_train_red_winged_blackbird)


shutil.rmtree('./data/datasets')

print('prepare dataset structure done.')

 - We define all necessary variables.

In [ ]:
path = './data/datasetsSelect'
train_path = './data/datasetsSelect/train'
test_path = './data/datasetsSelect/test'
val_path = './data/datasetsSelect/val'
IMG_SIZE = 100
scale = 1
batch_size = 10
nb_train_samples = 100
nb_validation_samples = 100
epochs = 10
labels = pd.read_csv("./data/classes.txt", nrows=10, names=["Classes"])


print(labels)



## Get trainning data

In [ ]:
def get_training_data(data_dir, size):
    data = list()
    for label in labels:
        path = os.path.join(data_dir, label)
        class_num = labels.index(label)
        for image in os.listdir(path):
            if(image != ".DS_Store"):
                try:
                    image_arr = cv2.imread(os.path.join(
                        path, image), cv2.IMREAD_GRAYSCALE)
                    resized_arr = cv2.resize(image_arr, (size, size))
                    data.append([resized_arr, class_num])
                except Exception as e:
                    print(e)
    return np.array(data, dtype="object")


In [ ]:
test = get_training_data(test_path, IMG_SIZE)
train = get_training_data(train_path, IMG_SIZE)
val = get_training_data(val_path, IMG_SIZE)

## Balance dataset


In [ ]:
def mix_datasets(array):
    arr_1 = array[np.where(array[:, 1] == 1)]
    arr_0 = array[np.where(array[:, 1] == 0)]
    l_arr_1 = len(arr_1)
    l_arr_0 = len(arr_0)
    diff = 0

    if l_arr_0 > l_arr_1:
        diff = l_arr_0 - l_arr_1
        arr_0.resize((l_arr_0 - diff, arr_0.shape[1]))
        array = np.concatenate((l_arr_0, arr_1))
    else:
        diff = l_arr_1 - l_arr_0
        arr_1.resize((l_arr_1 - diff, arr_1.shape[1]))
        array = np.concatenate((arr_0, arr_1))
    return array

## Scaling datasets

In [ ]:
train_scaled = train[np.random.randint(
    train.shape[0], size=int(len(train)*scale)), :]
test_scaled = test[np.random.randint(
    test.shape[0], size=int(len(test)*scale)), :]

## Preparing datasets

In [ ]:
def prepare_dataset(dataset):
    X_ = list()
    y_ = list()
    for features, label in dataset:
        X_.append(features)
        y_.append(label)
    X_ = np.array(X_)
    X_ = X_.reshape(-1, IMG_SIZE, IMG_SIZE, 1)
    y_ = np.array(y_)
    return X_, y_

In [ ]:
x_train, label_train = prepare_dataset(mix_datasets(train))
x_test, label_test = prepare_dataset(mix_datasets(test))
x_val, label_val = prepare_dataset(mix_datasets(val))

In [ ]:
# Append train + test + val data in order to scale our datasets
X = np.concatenate((x_train, x_test))
X = np.concatenate((X, x_val))
Y = np.concatenate((label_train, label_test))
Y = np.concatenate((Y, label_val))

In [ ]:
# Hold-Out cross validation
# test_size: what proportion of original data is used for test set 
x_train, x_test, label_train, label_test = train_test_split( X, Y, test_size=0.30)

Normalize Data

In [ ]:
xmax = np.array(x_train).max()
x_train = x_train / xmax
xmax = np.array(x_test).max()
x_test = x_test / xmax

### 2. Learning of model <a id="model"></a>

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPool2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras import layers, models

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu',
          input_shape=(IMG_SIZE, IMG_SIZE, 1)))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(120))
model.add(layers.Dense(2, activation='softmax'))

## Init training's Callback

In [ ]:
def callbacks():
    callbacks = [
        tf.keras.callbacks.ModelCheckpoint(
            filepath='lstmchar256256128test.h5',
            monitor='loss',
            save_best_only=True
        ),
        tf.keras.callbacks.EarlyStopping(
            monitor='loss',
            min_delta=0,
            patience=4,
            mode='auto',
            baseline=None,
        )
    ]
    return callbacks

In [ ]:
model.summary()

In [ ]:
y_test = np.array(label_test)
y_train = np.array(label_train)

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(
                  from_logits=True),
              metrics=['accuracy'])

time1 = time.time()
history = model.fit(x_train, y_train, epochs=20, validation_data=(
    x_test, y_test), callbacks=callbacks())

# Print duration time
duration_time = time.time() - time1
if duration_time < 60:
    print("{} s".format(duration_time))
else:
    print("{} min".format(duration_time/60))

# 3. Displaying metrics <a id="metric"></a>

## loss + accuracy graph

In [ ]:
import matplotlib.pyplot as plt

history_loss = history.history["loss"]
history_acc = history.history["accuracy"]

plt.plot(history_loss, label="loss")
plt.plot(history_acc, label="acc")
plt.title("Training history")
plt.legend()


## Prediction

In [ ]:
pred = model.predict(x_test)
print(pred)

In [ ]:
predicted_classes = np.argmax(pred, axis=1)
report = classification_report(predicted_classes, y_test, target_names=labels)
print(report)


We can see that our model obtains 85% precision for the guillemot pigeon against 81% for the Red headed woodpecker

### Cunsion Mactix

In [ ]:
score = model.evaluate(x_test, y_test, verbose=0)

print(f'Test loss     : {score[0]:4.4f}')
print(f'Test accuracy : {score[1]:4.4f}')
# print(classification_report(y_test, y_pred))

# conclusion <a id="conclusion"></a>

Based on the confusion matrix, we obtain an accuracy of 82% for the guillemot pigeon with a few proportion of false negatives , as well as an accuracy of 84% for the Red headed woodpecker with a small amount of false positive.

The cause of this discrepancy in accuracy can be due to several things such as an uneven distribution of our starting dataset with one class more present than the other.

In [ ]:
pickle.dump(model, open('model.sav', 'wb'))
